In [61]:
# Import packages for use
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import time
import psutil

#Connect to MySQL database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="0861137MySQL!",
  database="project1"
)

In [56]:
#Confirm connection has been established
print(mydb)

In [25]:
### LIGHT EXTRACTION ###
# Join the order_line and item tables and capture CPU, RAM and elapsed time to perform operations

#light extraction performance metrics
column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
lightExtractionPrfm = pd.DataFrame(columns = column_names)

#base metrics
print("\nBase CPU utilization: ", psutil.cpu_percent())
print("\nBase RAM utilization: ", psutil.virtual_memory().percent)

#Run 30 iterations to collect a df of performance metrics
for sampleNo in range(1):
    #script to pull data
    LEsql = "select * from item inner join order_line on order_line.orderedItem = item.orderedItem;"

    #Initiate timer for query
    start = time.perf_counter()
    print("Working on sample number: ", sampleNo+1,"\n")
    
    #insert data into dataFrame
    orderItemJoin = pd.read_sql(sql=LEsql, con=mydb)
    
    #Stop timer  
    stop = time.perf_counter()
    
    #load df with performance metrics    
    lightExtractionPrfm = lightExtractionPrfm.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                                   'RAM_utilization':  psutil.virtual_memory().percent,
                                                                   'elapsed_time': stop - start}, 
                                                                   index=[1]), ignore_index=True)
    time.sleep(5)
    
print("Data frame loading complete.")


Base CPU utilization:  1.5

Base RAM utilization:  19.1
Working on sample number:  1 

Data frame loading complete.


In [5]:
lightExtractionPrfm
#orderItemJoin.head(10)

,CPU_utilization,RAM_utilization,elapsed_time
0,2.4,23.6,12.675368
1,2.6,23.8,0.857316
2,2.6,23.8,0.867747
3,2.6,23.8,0.871417
4,2.3,23.8,0.871969
5,2.3,23.8,0.872208
6,2.9,23.8,0.872943
7,2.7,23.8,0.872611
8,2.4,23.8,0.877903
9,2.5,23.8,0.869302


In [ ]:
#Export DF to csv
lightExtractionPrfm.to_csv (r'D:\MSDS\2020 Spring\DB\Project\lightExtractionPerformance.csv', index = False, header=True)

In [26]:
### LIGHT TRANSFORMATION ###
# Preform light transformation workload.  In this case, identify all item descriptions with "Blue" in them 
# and change them to "Navy"

lightTrans = orderItemJoin
column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
lightTransPrfm = pd.DataFrame(columns = column_names)
#lightTrans.item_desc.str.contains("^Blue")

print("\nBase CPU utilization: ", psutil.cpu_percent())
print("\nBase RAM utilization: ", psutil.virtual_memory().percent)

#Run 30 iterations to collect transformation df of performance metrics
for sampleNoTransform in range(1):
    
    # Start Timer and progress tracker
    start = time.perf_counter()
    print("Working on sample number: ", sampleNoTransform+1,"\n")
    
    #Working Code
    lightTrans.item_desc = lightTrans.item_desc.str.replace('Blue', 'Navy', regex=True)
    #Stop timer  
    stop = time.perf_counter()
    
    
    lightTransPrfm = lightTransPrfm.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                         'RAM_utilization':  psutil.virtual_memory().percent,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)

    time.sleep(5)
print("Data frame loading complete.")


Base CPU utilization:  1.3

Base RAM utilization:  19.2
Working on sample number:  1 

Data frame loading complete.


In [27]:
lightTransPrfm
lightTrans = pd.DataFrame(lightTrans)
lightTrans

,orderedItem,item_desc,selling_price,Line,order,eaches_qty,orderedItem,POnum
0,024404P8,NavySkirtP8,46.61,1,436616,4,024404P8,840215
1,023806P6,YellowSlacksP6,52.83,2,436616,4,023806P6,840215
2,024408P6,PinkSkirtP6,46.61,3,436616,3,024408P6,840215
3,024409P6,OrangeSkirtP6,46.61,4,436616,9,024409P6,840215
4,024410P0,TanSkirtP0,46.61,5,436616,9,024410P0,840215
...,...,...,...,...,...,...,...,...
224481,023602P6,WhiteBlouseP6,18.76,224482,445860,4,023602P6,863021
224482,023404P10,NavyDressP10,87.87,224483,445860,2,023404P10,863021
224483,024205P8,GreenCasualJacketP8,68.34,224484,445860,15,024205P8,863021
224484,024004P2,NavyLeggingsP2,23.45,224485,445860,6,024004P2,863021


In [30]:
lightTrans = lightTrans.loc[:,~lightTrans.columns.duplicated()]
lightTrans

,orderedItem,item_desc,selling_price,Line,order,eaches_qty,POnum
0,024404P8,NavySkirtP8,46.61,1,436616,4,840215
1,023806P6,YellowSlacksP6,52.83,2,436616,4,840215
2,024408P6,PinkSkirtP6,46.61,3,436616,3,840215
3,024409P6,OrangeSkirtP6,46.61,4,436616,9,840215
4,024410P0,TanSkirtP0,46.61,5,436616,9,840215
...,...,...,...,...,...,...,...
224481,023602P6,WhiteBlouseP6,18.76,224482,445860,4,863021
224482,023404P10,NavyDressP10,87.87,224483,445860,2,863021
224483,024205P8,GreenCasualJacketP8,68.34,224484,445860,15,863021
224484,024004P2,NavyLeggingsP2,23.45,224485,445860,6,863021


In [64]:
### LIGHT LOADING ###
#Terminate connection from Extraction database, establish connection with Load database
mydb.close()

#Connect to datalake
dl = create_engine('mysql://{0}:{1}@{2}:{3}'.format('root', '0861137MySQL!', 'localhost', '3306')) 


ModuleNotFoundError: No module named 'MySQLdb'

In [58]:
#Confirm connection
print(datalake)

In [54]:
# Write dataframe to datalake

lightTrans.to_sql(name=datalake, con=dl, if_exists='replace', index=False)

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': Python type CMySQLConnection cannot be converted